<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Building a Real-time Recommendation API

This reference architecture shows the full lifecycle of building a recommendation system. It walks through the creation of appropriate azure resources, training a recommendation model using a Virtual Machine or Databricks, and deploying it as an API. It uses Azure Cosmos DB, Azure Machine Learning, and Azure Kubernetes Service. 

This architecture can be generalized for many recommendation engine scenarios, including recommendations for products, movies, and news. 
### Architecture
![architecture](https://camo.githubusercontent.com/35ffc0c1bb61f8cc928f02589941376d3005c82d/68747470733a2f2f7265636f64617461736574732e626c6f622e636f72652e77696e646f77732e6e65742f696d616765732f7265636f2d617263682e706e67 "Architecture")

**Scenario**: A media organization wants to provide movie or video recommendations to its users. By providing personalized recommendations, the organization meets several business goals, including increased click-through rates, increased engagement on site, and higher user satisfaction.

In this reference, we train and deploy a real-time recommender service API that can provide the top 10 movie recommendations for a given user. 

### Components
This architecture consists of the following key components:
* [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks)<sup>1)</sup> is used as a development environment to prepare input data and train the recommender model on a Spark cluster. Azure Databricks also provides an interactive workspace to run and collaborate on notebooks for any data processing or machine learning tasks.
* [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes)(AKS) is used to deploy and operationalize a machine learning model service API on a Kubernetes cluster. AKS hosts the containerized model, providing scalability that meets throughput requirements, identity and access management, and logging and health monitoring. 
* [Azure Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/introduction) is a globally distributed database service used to store the top 10 recommended movies for each user. Azure Cosmos DB is ideal for this scenario as it provides low latency (10 ms at 99th percentile) to read the top recommended items for a given user. 
* [Azure Machine Learning Service](https://docs.microsoft.com/en-us/azure/machine-learning/service/) is a service used to track and manage machine learning models, and then package and deploy these models to a scalable Azure Kubernetes Service environment.

<sup>1) Here, we are just describing an example of using Azure Databricks. Any platforms listed in [SETUP](https://github.com/microsoft/recommenders/blob/master/SETUP.md) can be used as well.</sup>


### Table of Contents.
0. [File Imports](#0-File-Imports)
1. [Service Creation](#1-Service-Creation)
2. [Training and evaluation](#2-Training)
3. [Operationalization](#3.-Operationalize-the-Recommender-Service)

## Setup
To run this notebook on Azure Databricks, you should import this notebook into your Azure Databricks Workspace first (see instructions [here](https://docs.azuredatabricks.net/user-guide/notebooks/notebook-manage.html#import-a-notebook)).

Setup for Azure Databricks should be completed by following the appropriate sections in the repository's [SETUP file](https://github.com/microsoft/recommenders/blob/master/SETUP.md).

Please note: This notebook **REQUIRES** that you add the dependencies to support **operationalization**. See the [SETUP](https://github.com/microsoft/recommenders/blob/master/SETUP.md) for details.


## 0 File Imports

In [1]:
import os
import sys
sys.path.append("../../")
import time
import urllib

from azure.common.client_factory import get_client_from_cli_profile
import azure.mgmt.cosmosdb
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice
from azureml.exceptions import WebserviceException
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import SparkPackage
import pydocumentdb.document_client as document_client
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType

from reco_utils.common.timer import Timer
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset import movielens
from reco_utils.dataset.cosmos_cli import find_collection, read_collection, read_database, find_database
from reco_utils.dataset.download_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from reco_utils.common.
print("Azure SDK version:", azureml.core.VERSION)

# Start spark session
cosmos_connector = (
    "https://search.maven.org/remotecontent?filepath=com/microsoft/azure/"
    "azure-cosmosdb-spark_2.3.0_2.11/1.3.3/azure-cosmosdb-spark_2.3.0_2.11-1.3.3-uber.jar"
)
jar_filepath = maybe_download(url=cosmos_connector, filename="cosmos.jar")
spark = start_or_get_spark("ALS", memory="10g", jars=[jar_filepath])
spark.sparkContext

display(sc)

PySpark version: 2.3.1
Azure SDK version: 1.0.69


## 1 Service Creation
Modify the **Subscription ID** to the subscription you would like to deploy to and set the resource name variables.

#### Services created by this notebook:
1. [Azure ML Service](https://azure.microsoft.com/en-us/services/machine-learning-service/)
1. [Azure Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/)
1. [Azure Container Registery](https://docs.microsoft.com/en-us/azure/container-registry/)
1. [Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/)
1. [Azure Application Insights](https://azure.microsoft.com/en-us/services/monitor/)
1. [Azure Storage](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-overview)
1. [Azure Key Vault](https://azure.microsoft.com/en-us/services/key-vault/)
1. [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/)

In [4]:
# Add your subscription ID
subscription_id = ""

# Resource group and workspace
resource_group = ""
workspace_name = ""

# Add your region to deploy Azure ML workspace
location = ""

Resource group: aml-111704
Worspace name: aml-ignite-ws-111704


In [ ]:
# CosmosDB
account_name = resource_group + "-ds-sql"
# account_name for CosmosDB cannot have "_" and needs to be less than 31 chars
account_name = account_name.replace("_", "-")[0:min(31, len(prefix))]
cosmos_database = "recommendations"
cosmos_collection = "user_recommendations_als"

# AzureML
model_name = "mvl-als-reco.mml"
service_name = "mvl-als"
aks_name = "mvl-als-aks"
container_image_name = "mvl-alk-img"

In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [5]:
userCol = "UserId"
itemCol = "MovieId"
ratingCol = "Rating"

train_data_path = "train"
test_data_path = "test"

### 1.1 Import or create the AzureML Workspace. 
This command will check if the AzureML Workspace exists or not, and will create the workspace if it doesn't exist.

In [6]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group, 
    location=location,
    exist_ok=True
)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


### 1.2 Create a Cosmos DB resource to store recommendation results:

In [7]:
# explicitly pass subscription_id in case user has multiple subscriptions
client = get_client_from_cli_profile(azure.mgmt.cosmosdb.CosmosDB, subscription_id=subscription_id)

async_cosmosdb_create = client.database_accounts.create_or_update(
    resource_group,
    account_name,
    {
        'location': location,
        'locations': [{
            'location_name': location
        }]
    }
)
account = async_cosmosdb_create.result()

my_keys = client.database_accounts.list_keys(resource_group, account_name)
master_key = my_keys.primary_master_key
endpoint = "https://" + account_name + ".documents.azure.com:443/"

# DB client
client = document_client.DocumentClient(endpoint, {'masterKey': master_key})

if not find_database(client, cosmos_database):
    db = client.CreateDatabase({'id': cosmos_database })
    print("Database created")
else:
    db = read_database(client, cosmos_database)
    print("Database found")

# Create collection options
options = dict(offerThroughput=11000)

# Create a collection
collection_definition = {'id': cosmos_collection, 'partitionKey': {'paths': ['/id'],'kind': 'Hash'}}
if not find_collection(client, cosmos_database, cosmos_collection):
    collection = client.CreateCollection(db['_self'], collection_definition, options)
    print("Collection created")
else:
    collection = read_collection(client, cosmos_database, cosmos_collection)
    print("Collection found")
    
dbsecrets = dict(
    Endpoint=endpoint, 
    Masterkey=master_key, 
    Database=cosmos_database, 
    Collection=cosmos_collection, 
    Upsert=True
)

Database found
Collection found


## 2 Training

Next, we train an [Alternating Least Squares model](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) on [MovieLens](https://grouplens.org/datasets/movielens/) dataset.

### 2.1 Download the MovieLens dataset

In [9]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", LongType()),
    )
)

data = movielens.load_spark_df(spark, size=MOVIELENS_DATA_SIZE, schema=schema)
data.show()

100%|██████████| 4.81k/4.81k [00:00<00:00, 14.5kKB/s]


+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



### 2.2 Split the data into train, test
There are several ways of splitting the data: random, chronological, stratified, etc., each of which favors a different real-world evaluation use case. We will split randomly in this example – for more details on which splitter to choose, consult [this guide](https://github.com/Microsoft/Recommenders/blob/master/notebooks/01_data/data_split.ipynb).

In [10]:
train, test = spark_random_split(data, ratio=0.75, seed=42)
print("N train", train.cache().count())
print("N test", test.cache().count())

N train 75031
N test 24969


### 2.3 Train the ALS model on the training data

To predict movie ratings, we use the rating data in the training set as users' explicit feedback. The hyperparameters used to estimate the model are set based on [this page](http://mymedialite.net/examples/datasets.html).

Under most circumstances, you would explore the hyperparameters and choose an optimal set based on some criteria. For additional details on this process, please see additional information in the deep dives [here](https://github.com/microsoft/recommenders/blob/master/notebooks/04_model_select_and_optimize/tuning_spark_als.ipynb).

In [11]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}

als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

In [12]:
model = als.fit(train)

### 2.4 Get top-k recommendations for our testing data

In the movie recommendation use case, recommending movies that have been rated by the users do not make sense. Therefore, the rated movies are removed from the recommended items.

In order to achieve this, we recommend all movies to all users, and then remove the user-movie pairs that exist in the training dataset.

In [13]:
# Get the cross join of all user-item pairs and score them.
users = train.select('UserId').distinct()
items = train.select('MovieId').distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)

dfs_pred.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|   148|    148| 2.7591963|
|   463|    148| 2.4122012|
|   471|    148|  4.056488|
|   496|    148| 2.3058019|
|   833|    148| 1.7793453|
|   243|    148|  2.541278|
|   392|    148|  2.538548|
|   540|    148| 3.4601738|
|   623|    148| 2.8626804|
|   737|    148| 1.9720266|
|   858|    148| 2.0933924|
|   897|    148| 3.7217906|
|    31|    148| 3.0197525|
|   516|    148| 3.5070436|
|    85|    148|  2.414331|
|   137|    148|  4.679284|
|   251|    148| 3.0501633|
|   451|    148|   4.08469|
|   580|    148| 3.6586523|
|   808|    148| 3.6865394|
+------+-------+----------+
only showing top 20 rows



In [14]:
# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train.alias("train"),
    (dfs_pred['UserId'] == train['UserId']) & (dfs_pred['MovieId'] == train['MovieId']),
    how='outer'
)
top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
    .select('pred.' + 'UserId', 'pred.' + 'MovieId', 'pred.' + "prediction")

top_all.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|     1|    587|  3.279942|
|     1|    869| 2.3982725|
|     1|   1208| 2.7008247|
|     1|   1677| 3.2902818|
|     2|     80| 2.6232195|
|     2|    303| 3.5138035|
|     2|    472| 2.9955308|
|     2|    582|  3.851469|
|     2|    838| 1.7562649|
|     2|    975| 2.6689043|
|     2|   1260| 4.2016172|
|     2|   1325|  1.174239|
|     2|   1381| 3.7950218|
|     2|   1530|  2.604762|
|     3|     22| 2.9423695|
|     3|     57| 1.6411926|
|     3|     89| 3.5102983|
|     3|    367|  2.226043|
|     3|   1091| 1.6633637|
|     3|   1167|  2.629972|
+------+-------+----------+
only showing top 20 rows



### 2.5 Evaluate how well ALS performs

Evaluate model performance using metrics such as Precision@K, Recall@K, [MAP](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\)) or [nDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain). For a full guide on what metrics to evaluate your recommender with, consult [this guide](https://github.com/Microsoft/Recommenders/blob/master/notebooks/03_evaluate/evaluation.ipynb).

In [15]:
test.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|     1|      2|   3.0|876893171|
|     1|      3|   4.0|878542960|
|     1|      4|   3.0|876893119|
|     1|     14|   5.0|874965706|
|     1|     17|   3.0|875073198|
|     1|     27|   2.0|876892946|
|     1|     29|   1.0|878542869|
|     1|     35|   1.0|878542420|
|     1|     36|   2.0|875073180|
|     1|     51|   4.0|878543275|
|     1|     52|   4.0|875072205|
|     1|     54|   3.0|878543308|
|     1|     56|   4.0|875072716|
|     1|     60|   5.0|875072370|
|     1|     64|   5.0|875072404|
|     1|     69|   3.0|875072262|
|     1|     77|   4.0|876893205|
|     1|     83|   3.0|875072370|
|     1|     85|   3.0|875073180|
|     1|     88|   4.0|878542791|
+------+-------+------+---------+
only showing top 20 rows



In [16]:
# Evaluate Ranking Metrics
rank_eval = SparkRankingEvaluation(
    test, 
    top_all, 
    k=TOP_K, 
    col_user="UserId", 
    col_item="MovieId", 
    col_rating="Rating", 
    col_prediction="prediction", 
    relevancy_method="top_k"
)

print(
    "Model:\tALS",
    "Top K:\t%d" % rank_eval.k,
    "MAP:\t%f" % rank_eval.map_at_k(),
    "NDCG:\t%f" % rank_eval.ndcg_at_k(),
    "Precision@K:\t%f" % rank_eval.precision_at_k(),
    "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n'
)

Model:	ALS
Top K:	10
MAP:	0.004037
NDCG:	0.036358
Precision@K:	0.038070
Recall@K:	0.013339


In [17]:
# Evaluate Rating Metrics
prediction = model.transform(test)
rating_eval = SparkRatingEvaluation(
    test, 
    prediction, 
    col_user="UserId", 
    col_item="MovieId", 
    col_rating="Rating", 
    col_prediction="prediction"
)

print(
    "Model:\tALS rating prediction",
    "RMSE:\t%.2f" % rating_eval.rmse(),
    "MAE:\t%f" % rating_eval.mae(),
    "Explained variance:\t%f" % rating_eval.exp_var(),
    "R squared:\t%f" % rating_eval.rsquared(), sep='\n'
)

Model:	ALS rating prediction
RMSE:	0.94
MAE:	0.738571
Explained variance:	0.293173
R squared:	0.288851


### 2.6 Save the model

In [20]:
(model
 .write()
 .overwrite()
 .save(model_name))

## 3. Operationalize the Recommender Service
Once the model is built with desirable performance, it will be operationalized to run as a REST endpoint to be utilized by a real time service. We will utilize [Azure Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/), [Azure Machine Learning Service](https://azure.microsoft.com/en-us/services/machine-learning-service/), and [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes) to operationalize the recommender service.

### 3.1 Create a look-up for Recommendations in Cosmos DB

First, the Top-10 recommendations for each user as predicted by the model are stored as a lookup table in Cosmos DB. At runtime, the service will return the Top-10 recommendations as precomputed and stored in Cosmos DB:

In [19]:
recs = model.recommendForAllUsers(10)
recs_topk = recs.withColumn("id",recs[userCol].cast("string")) \
    .select("id", "recommendations."+ itemCol).toPandas()
recs_topk.head()

,id,MovieId
0,471,"[898, 336, 1005, 1160, 263, 888, 375, 1061, 34..."
1,463,"[1449, 511, 1064, 958, 1512, 408, 750, 272, 16..."
2,833,"[1203, 1643, 320, 1512, 1368, 390, 853, 641, 6..."
3,496,"[593, 968, 960, 130, 915, 902, 1022, 580, 512,..."
4,148,"[1104, 960, 580, 1069, 1126, 1159, 593, 718, 5..."


In [21]:
# Save data to CosmosDB
(spark.createDataFrame(recs_topk)
 .write
 .format("com.microsoft.azure.cosmosdb.spark")
 .mode('overwrite')
 .options(**dbsecrets)
 .save())

### 3.2 Configure Azure Machine Learning

Next, Azure Machine Learning Service is used to create a model scoring image and deploy it to Azure Kubernetes Service as a scalable containerized service. To achieve this, a **scoring script** and an **environment config** should be created. The following shows the content of the two files.  

In the scoring script, we make a call to Cosmos DB to lookup the top 10 movies to recommend given an input User ID:

In [22]:
score_sparkml = """
import json
import pydocumentdb.document_client as document_client

def init(local=False):
    global client, collection
    try:
      client = document_client.DocumentClient('{endpoint}', dict(masterKey='{key}'))
      collection = client.ReadCollection(collection_link='dbs/{database}/colls/{collection}')
    except Exception as e:
      collection = e

def run(input_json):
    try:
      # Query them in SQL
      id = str(json.loads(json.loads(input_json)[0])['id'])
      query = dict(query='SELECT * FROM c WHERE c.id = "' + id +'"')
      options = dict(partitionKey=str(id))
      document_link = 'dbs/{database}/colls/{collection}/docs/' + id
      result = client.ReadDocument(document_link, options);  
    except Exception as e:
        result = str(e)
    return json.dumps(str(result))
""".format(key=dbsecrets['Masterkey'], 
           endpoint=dbsecrets['Endpoint'], 
           database=dbsecrets['Database'], 
           collection=dbsecrets['Collection'])

# test validity of python string
exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

Register your model:

In [23]:
mymodel = Model.register(model_path=model_name,  # this points to a local file
                         model_name=model_name,  # this is the name the model is registered as, currently same name for both path and name                 
                         description="AML trained model",
                         workspace=ws)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model mvl-als-reco.mml
mvl-als-reco.mml AML trained model 3


### 3.3 Deploy the model as a Service on AKS

#### 3.3.1 Create an Environment for your model:

In [24]:
env = Environment(name='sparkmlenv')

# Specify a public image from microsoft/mmlspark as base image
env.docker.base_image="microsoft/mmlspark:0.15"

pip = ['azureml-defaults', 'numpy==1.14.2', 'scikit-learn==0.19.1', 'pandas', 'pydocumentdb']

# Add dependencies needed for inferencing
env.python.conda_dependencies = CondaDependencies.create(pip_packages=pip)
env.inferencing_stack_version = "latest"

# Add spark packages
env.spark.precache_packages = True
env.spark.repositories = ["https://mmlspark.azureedge.net/maven"]
env.spark.packages= [SparkPackage("com.microsoft.ml.spark", "mmlspark_2.11", "0.15"),
                     SparkPackage("com.microsoft.azure", artifact="azure-storage", version="2.0.0"),
                     SparkPackage(group="org.apache.hadoop", artifact="hadoop-azure", version="2.7.0")]

#### 3.3.2 Create an AKS Cluster to run your container (this may take 20-25 minutes):

In [25]:
# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create the cluster using the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(workspace=ws, 
                                      name=aks_name, 
                                      provisioning_configuration=prov_config)
    aks_target.wait_for_completion(show_output = True)
    print(aks_target.provisioning_state)
    # To check any error logs, print(aks_target.provisioning_errors)

Found existing cluster, use it.


#### 3.3.3 Deploy the container image to AKS:

In [26]:
# Create an Inferencing Configuration with your environment and scoring script
inference_config = InferenceConfig(environment=env,
                                   entry_script="score_sparkml.py")

# Set the web service configuration (using default here with app insights)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

# Webservice creation using single command
try:
    aks_service = Model.deploy(workspace=ws,
                               models=[mymodel],
                               name=service_name,
                               inference_config=inference_config,
                               deployment_config=aks_config,
                               deployment_target=aks_target)
    aks_service.wait_for_deployment(show_output=True)
except WebserviceException:
    # Retrieve existing service.
    aks_service = Webservice(ws, name=service_name)
    print("Retrieved existing service")

Running................................................................................................................
SucceededAKS service creation operation finished, operation "Succeeded"


### 3.4 Call the AKS model service
After the deployment, the service can be called with a user ID – the service will then look up the top 10 recommendations for that user in Cosmos DB and send back the results.
The following script demonstrates how to call the recommendation service API and view the result for the given user ID:

In [27]:
scoring_url = aks_service.scoring_uri
service_key = aks_service.get_keys()[0]

input_data = '["{\\"id\\":\\"496\\"}"]'.encode()

req = urllib.request.Request(scoring_url,data=input_data)
req.add_header("Authorization","Bearer {}".format(service_key))
req.add_header("Content-Type","application/json")

with Timer() as t: 
    with urllib.request.urlopen(req) as result:
        res = result.readlines()
        print(res)
    
print("Full run took %.2f seconds" % t.interval)

[b'"\\"{\'MovieId\': [593, 968, 960, 130, 915, 902, 1022, 580, 512, 721], \'id\': \'496\', \'_rid\': \'1r1rALvGnC3LAAAAAAAACA==\', \'_self\': \'dbs/1r1rAA==/colls/1r1rALvGnC0=/docs/1r1rALvGnC3LAAAAAAAACA==/\', \'_etag\': \'\\\\\\"66008bff-0000-0800-0000-5dc1f4d50000\\\\\\"\', \'_attachments\': \'attachments/\', \'_ts\': 1572992213}\\""']
Full run took 0.12 seconds
